In [ ]:
import sys
import pandas as pd
import numpy as np

data = pd.read_csv('../../../data/hw1/train.csv')

# **Preprocessing** 
取需要的數值部分，从第3列开始截取，將 'RAINFALL' 字段的'NR'替换为0

In [ ]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

# **Extract Features (1)**
將原始 4320 * 24 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料.  
下图为一个月数据的转换，把每个月数据转好，放到month_data[12]中
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/2bcba0f134310b17fa4fde18e68f5cf5.jpg)

In [ ]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

# **Extract Features (2)**
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p5.png)
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p6.png)

每個月會有 480hrs，每 10 小時形成一组 data 与 answer（第10个小时为一个answer）  
每個月會有 471 组 data（第1组：[1 2..10]  第2组[2 3..11] .. 第471组[471 472..480]）  
故總資料數為 471 * 12 组，而每组中 data 有 9 * 18 个features (一小時 18 個 features * 9 小時)，
target 則也有18個(第 10 個小時的 PM2.5)

In [ ]:
#x: 471组/月 * 12月  ,18个特征*9小时
x = np.empty([12 * 471, 18 * 9], dtype = float)
#y: 471组/月 * 12月, 1个结果(pm2.5)
y = np.empty([12 * 471, 1], dtype = float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
                
            # month_data[month] => 18x480   (18x9).reshape(1,-1) => (1x162)
            x[month * 471 + day * 24 + hour, :] = \
                month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            
            # month_data[month][9, day * 24 + hour + 9]  9表示第10行为pm2.5，并取第10小时的值
            y[month * 471 + day * 24 + hour, 0] = \
                month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

# **Normalize (1)**


In [28]:
#x:[12 * 471, 18 * 9] axis=0 同列相加求平均 结果数=列数
mean_x = np.mean(x, axis = 0) 
#x:[12 * 471, 1] axis=0 算同列数据 标准差， 结果数=列数
std_x = np.std(x, axis = 0) #18 * 9 

print(mean_x.shape)
print(std_x.shape)

for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

(162,)


IndexError: invalid index to scalar variable.

#**Split Training Data Into "train_set" and "validation_set"**
這部分是針對作業中 report 的第二題、 第三題做的簡單示範，以生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

# **Training**
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p8.png)
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p9.png)
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p10.png)


(和上圖不同處: 下面的 code 採用 Root Mean Square Error)

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [ ]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

# **Testing**
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p11.png)

載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

In [ ]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('../../../data/hw1/test.csv', header = None)
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

# **Prediction**
說明圖同上

![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p11.png)

有了 weight 和測試資料即可預測 target。

In [ ]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

# **Save Prediction to CSV File**


In [ ]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

相關 reference 可以參考:

Adagrad :
https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705 

RMSprop : 
https://www.youtube.com/watch?v=5Yt-obwvMHI 

Adam
https://www.youtube.com/watch?v=JXQT_vxqwIs 


以上 print 的部分主要是為了看一下資料和結果的呈現，拿掉也無妨。另外，在自己的 linux 系統，可以將檔案寫死的的部分換成 sys.argv 的使用 (可在 terminal 自行輸入檔案和檔案位置)。

最後，可以藉由調整 learning rate、iter_time (iteration 次數)、取用 features 的多寡(取幾個小時，取哪些特徵欄位)，甚至是不同的 model 來超越 baseline。

Report 的問題模板請參照 : https://docs.google.com/document/d/1s84RXs2AEgZr54WCK9IgZrfTF-6B1td-AlKR9oqYa4g/edit